# Data Loading 

### Car Price Prediction Data 

This data contains information on car prices measured in 2023. As we are observing EVs we were required to filter out hybrid, petrol and diesel powertrains. 

The dataset contained the price of vehicles new and used. We used this to match MSRP of vehicles to their advertised used price to gain insight on residual values. 

The dataset when filtered to used electric vehicles was small at 214 data points. We will have to consider this when using machine learning methods but electric vehicle used markets behave contrastingly to other power trains and this signifies the importance of keeping the data as aligned to reality as possible.

In [72]:
# Importing libraries 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [73]:
df = pd.read_csv("Data/car_price_prediction_.csv", index_col=0)

# Data Loading 

In [74]:
df.head()

,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price,Model
Car ID,,,,,,,,,
1,Tesla,2016,2.3,Petrol,Manual,114832,New,26613.92,Model X
2,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series
3,Audi,2013,4.5,Electric,Manual,181601,New,44402.61,A4
4,Tesla,2011,4.1,Diesel,Automatic,68682,New,86374.33,Model Y
5,Ford,2009,2.6,Diesel,Manual,223009,Like New,73577.10,Mustang


In [75]:
# Filtering used EVs
used_ev = df[(df['Fuel Type'] == 'Electric') & (df['Condition'] == 'Used')].copy()

# Filtering new EVs
new_ev = df[(df['Fuel Type'] == 'Electric') & (df['Condition'] == 'New')].copy()



In [76]:
# Keeping only one new car per Brand + Model (to avoid duplicates)
new_ev_unique = new_ev.drop_duplicates(subset=['Brand', 'Model'])[['Brand', 'Model', 'Price']]


In [77]:
# matching new prices to used sales 
used_ev_matched = used_ev.merge(
    new_ev_unique,
    on=['Brand', 'Model'],
    how='left',  
    suffixes=('_Used', '_New')
)


In [78]:
# Filtering to perfect matches 
perfect_matches = used_ev_matched[used_ev_matched['Price_New'].notna()]

print(f"Number of perfect matches: {perfect_matches.shape[0]}")


Number of perfect matches: 214


In [79]:
perfect_matches

,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price_Used,Model,Price_New
0,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series,28992.68
1,Audi,2020,4.0,Electric,Automatic,135486,Used,63498.75,Q5,82480.40
2,Ford,2017,5.7,Electric,Automatic,169737,Used,28074.19,Mustang,25179.40
3,Ford,2010,3.9,Electric,Automatic,240904,Used,21796.16,Mustang,25179.40
4,Mercedes,2022,2.3,Electric,Manual,12150,Used,61393.26,E-Class,45314.56
...,...,...,...,...,...,...,...,...,...,...
209,Toyota,2023,3.4,Electric,Automatic,148940,Used,89389.53,RAV4,45764.29
210,Ford,2014,5.6,Electric,Manual,120926,Used,78269.05,Explorer,40085.49
211,Toyota,2004,3.2,Electric,Automatic,157234,Used,78647.35,Prius,26512.16
212,Toyota,2001,1.5,Electric,Manual,14407,Used,22705.09,Prius,26512.16


In [80]:
perfect_matches['Residual_Percentage'] = (
    perfect_matches['Price_Used'] / perfect_matches['Price_New'] * 100
)


In [81]:
perfect_matches

,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price_Used,Model,Price_New,Residual_Percentage
0,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series,28992.68,50.632125
1,Audi,2020,4.0,Electric,Automatic,135486,Used,63498.75,Q5,82480.40,76.986472
2,Ford,2017,5.7,Electric,Automatic,169737,Used,28074.19,Mustang,25179.40,111.496660
3,Ford,2010,3.9,Electric,Automatic,240904,Used,21796.16,Mustang,25179.40,86.563461
4,Mercedes,2022,2.3,Electric,Manual,12150,Used,61393.26,E-Class,45314.56,135.482414
...,...,...,...,...,...,...,...,...,...,...,...
209,Toyota,2023,3.4,Electric,Automatic,148940,Used,89389.53,RAV4,45764.29,195.325941
210,Ford,2014,5.6,Electric,Manual,120926,Used,78269.05,Explorer,40085.49,195.255316
211,Toyota,2004,3.2,Electric,Automatic,157234,Used,78647.35,Prius,26512.16,296.646331
212,Toyota,2001,1.5,Electric,Manual,14407,Used,22705.09,Prius,26512.16,85.640287


In [82]:
# Checking how many imperfect matches were made
unmatched = used_ev_matched[used_ev_matched['Price_New'].isna()]
print(f"Number of unmatched used EVs: {unmatched.shape[0]}")

# Check first few matches
perfect_matches.head()


Number of unmatched used EVs: 0


,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price_Used,Model,Price_New,Residual_Percentage
0,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series,28992.68,50.632125
1,Audi,2020,4.0,Electric,Automatic,135486,Used,63498.75,Q5,82480.40,76.986472
2,Ford,2017,5.7,Electric,Automatic,169737,Used,28074.19,Mustang,25179.40,111.496660
3,Ford,2010,3.9,Electric,Automatic,240904,Used,21796.16,Mustang,25179.40,86.563461
4,Mercedes,2022,2.3,Electric,Manual,12150,Used,61393.26,E-Class,45314.56,135.482414


## Summary


We now have a smaller dataset - 214 rows, but this is much more useful for our analysis and allows us to see residual values of the cars related to our investigations.